# Load books

In [1]:
import glob
import os
books_directory = '/home/ubuntu/Downloads/twiker-experiments/process_analysis/Stephen/books'
books = []

# List all .txt files in the directory using glob
book_files = glob.glob(os.path.join(books_directory, '*.txt'))
# Iterate over each file in the directory
for book_file in book_files:
    with open(book_file, 'r',encoding='UTF-8') as fs:
        books.append(fs.read())
    print(f'{os.path.basename(book_file)}', len(books[-1]))
print()
#print(books[0][0:1000])

2012 - A Face in the Crowd.txt 109317
2001 - Dreamcatcher.txt 1229984
1982 - The Running Man.txt 400312
1991 - Needful Things.txt 1391839
2010 - Full Dark No Stars.txt 729712
1984 - Thinner.txt 535048
1983 - Cycle of the Werewolf.txt 88213
1983 - Pet Sematary.txt 795603
2004 - Song of Susannah.txt 707473
1996 - Desperation.txt 1065799
2005 - The Colorado Kid.txt 200947
1998 - Bag of Bones.txt 1091002
2008 - Duma Key.txt 1114311
1986 - It.txt 2457994
1999 - Hearts In Atlantis.txt 1013064
1995 - Rose Madder.txt 960452
1992 - Geralds Game.txt 693212
2004 - The Dark Tower.txt 1528909
2010 - UR.txt 120843
1981 - The Mist.txt 278170
2008 - Just After Sunset.txt 741047
2012 - In the Tall Grass.txt 108462
1987 - The Drawing Of The Three.txt 712126
1980 - Firestarter.txt 845812
1996 - The Regulators.txt 652027
2007 - Blaze.txt 453823
2011 - 11_22_63.txt 1485832
1975 - Salems Lot.txt 1071564
2006 - Cell.txt 676291
1984 - The Talisman.txt 1515080
1977 - Rage.txt 292880
2002 - Everythings Eventual

In [2]:
import re
def clean_text(text):
    """Clean the text by removing special characters, tags, etc."""
    # Replace newlines and carriage returns with a space
    #text = text.replace('\n', ' ')
    #text = text.replace('\n\n', '\n')
    # Remove all non-alphanumeric characters except spaces
    #text = re.sub(r'[^A-Za-z0-9\s]+', ' ', text)
    #text = re.sub('<[^>]+>%/', ' ', text)
    #text = re.sub('<[^>]+>', '', text) # Remove any character that is not a letter, number, punctuation, or whitespace 
    #text = re.sub(r'[^A-Za-z0-9\s.,!?;:()-]', '', text)
    text = re.sub(r'\.{3,}\s*', '.', text)
    # Remove leading tabs and extra spaces
    #text = re.sub(r'\s+', ' ', text).strip()
    return text

cleaned_books =[clean_text(book) for book in books]

In [11]:
# Some long paragraphs will be truncated during training. 
# To avoid lossing data, we split them into shorter ones.

import nltk
from transformers import GPT2Tokenizer

# Download the necessary NLTK data
nltk.download('punkt')

# Initialize the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

def split_text_at_sentences(text, max_tokens=4000):
    # Tokenize the text into sentences
    sentences = nltk.tokenize.sent_tokenize(text)
    
    chunks = []
    current_chunk = []
    current_tokens = []
    
    for sentence in sentences:
        # Tokenize the sentence
        sentence_tokens = tokenizer.encode(sentence, add_special_tokens=False)
        
        # Check if adding this sentence would exceed the max token limit
        if len(current_tokens) + len(sentence_tokens) > max_tokens:
            # If so, finalize the current chunk and start a new one
            chunks.append(' '.join(current_chunk))
            current_chunk = []
            current_tokens = []
        
        # Add the sentence to the current chunk
        current_chunk.append(sentence)
        current_tokens.extend(sentence_tokens)
    
    # Add the last chunk if there's any content left
    if current_chunk:
        chunks.append(' '.join(current_chunk))
    
    return chunks

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
chunked_books = []
for i, book in enumerate(cleaned_books):
    chunked_book = ''
    book='\n'.join(line for line in book.splitlines() if line.strip())
    chunks = split_text_at_sentences(book.replace("\n", " "))
    for chunk in chunks:
        chunked_book += chunk + "\n"
    chunked_books.append(chunked_book)

Token indices sequence length is longer than the specified maximum sequence length for this model (1199 > 1024). Running this sequence through the model will result in indexing errors


# Merge to train and eval

In [23]:
num_train_books = 17
train = ''
eval =''
for i in range(num_train_books):
    train += chunked_books[i]
for j in range(len(chunked_books)-num_train_books):    
    eval += chunked_books[num_train_books+j]

with open(f'data_files/train.txt', 'w') as fs:
    fs.write(train)

with open(f'data_files/eval.txt', 'w') as fs:
    fs.write(eval)

with open(f'data_files/full.txt', 'w') as fs:
    fs.write(train + eval)

In [24]:
def count_lines_in_file(filename):
    with open(filename, 'r') as file:
        line_count = sum(1 for line in file)
    return line_count
 
# Example usage:
filename = 'your_text_file.txt'
train_number_of_lines = count_lines_in_file('data_files/train.txt')
full_number_of_lines = count_lines_in_file('data_files/full.txt')
eval_number_of_lines = count_lines_in_file('data_files/eval.txt')
print(f'The train file has {train_number_of_lines} lines.')
print(f'The full file has {full_number_of_lines} lines.')
print(f'The eval file has {eval_number_of_lines} lines.')

The train file has 941 lines.
The full file has 3588 lines.
The eval file has 2647 lines.
